# 🚀 PiEdge EduKit - Snabbkörning & Sanity Check

**Mål**: Kör hela lektionen snabbt och se att allt fungerar.

Detta notebook kör samma steg som `run_lesson.sh`, men som klickbara celler. Perfekt för att:
- Komma igång snabbt
- Se helheten innan vi går in på detaljer
- Verifiera att din miljö fungerar

> **💡 Tips**: Kör cellerna i ordning. Om något går fel, kolla att du har Python 3.12 och att `.venv` är aktiverat.


## 1️⃣ Setup & Verifiering

Först kontrollerar vi att miljön är korrekt:


In [ ]:
# Quiet mode for cleaner outputs in Jupyter
import os, warnings
os.environ.setdefault("TQDM_DISABLE", "1")  # hide tqdm progress bars
warnings.filterwarnings("ignore", category=UserWarning, module="onnxruntime")



In [1]:
# Miljökoll + självläkning (Python 3.12 + editable install)
import sys, os, importlib, subprocess
print(f"Python version: {sys.version}")
assert sys.version_info[:2] == (3, 12), f"Python 3.12 krävs, du har {sys.version_info[:2]}"

try:
    import piedge_edukit  # noqa: F401
    print("✅ PiEdge EduKit package OK")
except ModuleNotFoundError:
    # Hitta repo-roten: om vi står i labs/, gå ett steg upp
    repo_root = os.path.abspath(os.path.join(os.getcwd(), "..")) if os.path.basename(os.getcwd()) == "labs" else os.getcwd()
    print("⚠ Package saknas – installerar editable från:", repo_root)
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-e", repo_root])
    importlib.invalidate_caches()
    import piedge_edukit  # noqa: F401
    print("✅ Package installerat")


Python version: 3.12.10 (tags/v3.12.10:0cc8128, Apr  8 2025, 12:21:36) [MSC v.1943 64 bit (AMD64)]
✅ PiEdge EduKit package OK


In [2]:
# Paketet ska redan vara installerat av cellen ovan. Enkel sanity:
import piedge_edukit
print("✅ Paketet importeras – kör vidare!")


✅ Paketet importeras – kör vidare!


## 2️⃣ Träning & ONNX Export

Tränar en liten modell med FakeData och exporterar till ONNX:


In [3]:
# Träna modell (snabb körning för demo)
!python -m piedge_edukit.train --fakedata --no-pretrained --epochs 1 --batch-size 256 --output-dir ./models


Using device: cpu
[INFO] Using FakeData for training (no real images)
Preparing data...
Set 2 classes: ['class0', 'class1']
Training model with 2 classes...
Classes: ['class0', 'class1']

Epoch 1/1
Train Loss: 0.7062, Train Acc: 51.00%
Val Loss: 0.6809, Val Acc: 65.00%

Training completed! Best validation accuracy: 65.00%
Exporting model to ONNX...
[OK] Model exported to models\model.onnx (opset 17)
[OK] ONNX model verified successfully
  Input shape: (1, 3, 64, 64)
  Output shape: (1, 2)
  Output dtype: float32
[OK] Preprocessing configuration valid (hash: 9f9a96cb4a32eea9)
[OK] Labels valid: 2 classes
[OK] Model exported successfully to models\model.onnx
[OK] Training and export completed successfully!



Training: 100%|##########| 1/1 [00:02<00:00,  2.03s/it]

Validation: 100%|##########| 1/1 [00:01<00:00,  1.64s/it]


In [4]:
# Kontrollera att modellen skapades
import os
if os.path.exists("./models/model.onnx"):
    size_mb = os.path.getsize("./models/model.onnx") / (1024*1024)
    print(f"✅ ONNX-modell skapad: {size_mb:.1f} MB")
else:
    print("❌ ONNX-modell saknas")


✅ ONNX-modell skapad: 8.5 MB


## 3️⃣ Latensbenchmark

Mäter hur snabb modellen är på CPU:


In [ ]:
# Kör benchmark (snabb körning)
!python -m piedge_edukit.benchmark --fakedata --model-path ./models/model.onnx --warmup 1 --runs 3 --providers CPUExecutionProvider


In [ ]:
# Visa benchmark-resultat
if os.path.exists("./reports/latency_summary.txt"):
    with open("./reports/latency_summary.txt", "r") as f:
        print("📊 Benchmark-resultat:")
        print(f.read())
else:
    print("❌ Benchmark-rapport saknas")


## 4️⃣ Kvantisering (INT8)

Komprimerar modellen för snabbare inference:


In [ ]:
# Kör kvantisering
!python -m piedge_edukit.quantization --fakedata --model-path ./models/model.onnx --calib-size 16


In [ ]:
# Visa kvantiseringsresultat
if os.path.exists("./reports/quantization_summary.txt"):
    with open("./reports/quantization_summary.txt", "r") as f:
        print("⚡ Kvantiseringsresultat:")
        print(f.read())
else:
    print("❌ Kvantiseringsrapport saknas")


## 5️⃣ Utvärdering & Verifiering

Testar modellen och genererar kvitto:


In [ ]:
# Kör utvärdering
!python scripts/evaluate_onnx.py --model ./models/model.onnx --fakedata --limit 16


In [ ]:
# Kör verifiering och generera kvitto
!python verify.py


In [ ]:
# Visa kvitto
import json
if os.path.exists("./progress/receipt.json"):
    with open("./progress/receipt.json", "r") as f:
        receipt = json.load(f)
    print("📋 Verifieringskvitto:")
    print(f"Status: {'✅ PASS' if receipt['pass'] else '❌ FAIL'}")
    print(f"Timestamp: {receipt['timestamp']}")
    print("\nKontroller:")
    for check in receipt['checks']:
        status = "✅" if check['ok'] else "❌"
        print(f"  {status} {check['name']}: {check['reason']}")
else:
    print("❌ Kvitto saknas")


## 🎉 Klar!

Du har nu kört hela PiEdge EduKit-lektionen! 

**Nästa steg**: Gå till `01_training_and_export.ipynb` för att förstå vad som hände under träningen.

**Genererade filer**:
- `models/model.onnx` - Tränad modell
- `reports/` - Benchmark och kvantiseringsrapporter
- `progress/receipt.json` - Verifieringskvitto
